In [ ]:
import pathlib as pl
import sys
import json
import datetime as dt
import textwrap
import os

import pandas as pd
import dotenv

sys.path.append("../")
import payulator as pu

%load_ext autoreload
%autoreload 2

dotenv.load_dotenv("../.env")

In [ ]:
ROOT = pl.Path.home() / "merriweather/loans/active" 

%ls {ROOT}
%ls {ROOT / "C-Vallyon-20220201" }

In [ ]:
import smtplib
import ssl

from email import encoders
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.utils import formatdate


SENDER = "alex@merriweather.nz"
SMTP_SERVER = "mail.gandi.net"
PORT = 465  # for SSL
PASSWORD = os.getenv("PASSWORD")

def send_email(
    subject, 
    body, 
    recipients_to,
    recipients_cc=None, 
    recipients_bcc=None, 
    attachment_path=None,
    sender=SENDER,
    smtp_server=SMTP_SERVER,
    port=PORT, 
    password=PASSWORD,
):
    """
    Send an email with an attachment.
    """
    # Make the message
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["Subject"] = subject
    msg["Date"] = formatdate(localtime=True)
    if body:
        msg.attach( MIMEText(body) )

    msg["To"] = ', '.join(recipients_to)
    if recipients_cc is None:
        recipients_cc = []
    if recipients_bcc is None:
        recipients_bcc = []
        
    msg["cc"] = ', '.join(recipients_cc)
    msg["bcc"] = ', '.join(recipients_bcc)
        
    if attachment_path is not None:
        attachment_path = pl.Path(attachment_path)
        header = "Content-Disposition", f"attachment; filename={attachment_path.name}"
        attachment = MIMEBase('application', "octet-stream")
        try:
            with attachment_path.open() as src:
                data = src.read()
            attachment.set_payload(data)
            encoders.encode_base64(attachment)
            attachment.add_header(*header)
            msg.attach(attachment)
        except IOError:
            raise IOError(f"Error opening attachment file {attachment_path}")

    recipients = recipients_to + recipients_cc + recipients_bcc

    # Login to server and send mail
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender, password)
        server.sendmail(sender, recipients, msg.as_string())

# Email latest payment summary for each active loan

In [ ]:
do_real_run = True

for parent in sorted(ROOT.iterdir()):
    if not parent.is_dir():
        continue
        
    print("-"*40)

    # Get loan parameters
    with (parent / "parameters.json").open() as src:
        params = json.load(src)

    loan_code = params["code"]
    first_payment_date = pd.to_datetime(params["first_payment_date"])
    today = dt.datetime.now()

    # Skip if no payments due yet
    if first_payment_date > today:
        print(f"Skipping loan {loan_code} because no payments due yet")
        continue

    # Build email
    borrower_email = params["borrower_email"]
    attachment_path = parent / "payment_schedule.csv"        
    body = textwrap.dedent(f"""
        Greetings from Merriweather.

        Attached for your records is the latest payment summary* for your loan {loan_code}.  
        Enjoy.
        
        Alex Raichev
        Director of Merriweather
        https://merriweather.nz        

        [*] The `total_payment` column is sometimes wrong by a cent (compared to the loan contract specification),
        a rounding error you can ignore.
        """
    )


    if do_real_run:
        print(f"Emailing {borrower_email} re loan {loan_code}...")
        send_email(
            subject=f"Payment summary for loan {loan_code}",
            body=body,
            recipients_to=[borrower_email],
            recipients_bcc=["alex@merriweather.nz"],
            attachment_path=attachment_path,
        )
    else:
        print(f"Emailing hello@paperdragon.nz re loan {loan_code}...")
        send_email(
            subject=f"Payment summary for loan {loan_code}",
            body=body,
            recipients_to=["hello@paperdragon.nz"],
            recipients_bcc=["alex@merriweather.nz"],
            attachment_path=attachment_path,
        )
